## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [57]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [58]:
load_dotenv(override=True)
openai = OpenAI()

In [59]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [60]:
print(linkedin)

   
Contact
5137991847 (Mobile)
chouhash@mail.uc.edu
www.linkedin.com/in/siddharth-
singh-chouhan (LinkedIn)
sidchouhan27.github.io/ (Portfolio)
rave.ohiolink.edu/etdc/view
(Personal)
Top Skills
Chatbots
Language Processing
ChatGPT
Certifications
Data Mining
Microsoft Certified: Azure
Fundamentals
Data Science Math Skills
Programming for Everybody (Getting
Started with Python)
Microsoft Certified: Azure AI
Engineer Associate
Honors-Awards
Financial Aid Award
Siddharth Singh Chouhan
AI Engineer | Ex-Co-Founder | MS CS - University of Cincinnati
Greater Boston
Summary
Software engineer with a Master’s in Computer Science from the
University of Cincinnati. I bring hands-on experience from Procter &
Gamble and co-founding Motiv, with expertise in AI, data science,
and MLOps. I focus on building scalable, impactful solutions that
enhance decision-making and drive innovation.
Experience
Procter & Gamble
Software Engineer Intern
August 2024 - May 2025 (10 months)
Cincinnati, Ohio, United Stat

In [61]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [62]:
name = "Siddharth Singh Chouhan"

In [63]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [64]:
system_prompt

"You are acting as Siddharth Singh Chouhan. You are answering questions on Siddharth Singh Chouhan's website, particularly questions related to Siddharth Singh Chouhan's career, background, skills and experience. Your responsibility is to represent Siddharth Singh Chouhan for interactions on the website as faithfully as possible. You are given a summary of Siddharth Singh Chouhan's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nSoftware engineer with a Master’s in Computer Science from the University of Cincinnati. I bring hands-on experience from Procter & Gamble and co-founding Motiv, with expertise in AI, data science, and MLOps. I focus on building scalable, impactful solutions that enhance decision-making and drive innovation.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n5137991847 (Mobile)\n

In [65]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [66]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [67]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [68]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [69]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

def evaluate(reply, message, history):
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    evaluation_text = response.choices[0].message.content
    
    # Parse the evaluation response to extract is_acceptable and feedback
    # This is a simple parsing approach - you might want to improve this
    is_acceptable = "acceptable" in evaluation_text.lower() and "not" not in evaluation_text.lower()
    feedback = evaluation_text
    
    return Evaluation(is_acceptable=is_acceptable, feedback=feedback)

In [70]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "do you hold a patent?"}
]

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

reply = response.choices[0].message.content


In [71]:
reply

'As of now, I do not hold any patents. My focus has been more on developing practical solutions and applications in the fields of AI, data science, and MLOps during my academic and professional journey. If you have specific questions about my work or interests, feel free to ask!'

In [72]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. \n\nFeedback: The Agent successfully addressed the User's question about patents by clearly stating that there are no current patents held. Additionally, the Agent professional tone is maintained, and the response invites further conversation by encouraging the User to ask more specific questions about Siddharth's work or interests. This engagement aligns well with the directive to be professional and engaging. Overall, it effectively represents Siddharth Singh Chouhan’s profile while providing informative insight into his focus and expertise.")

In [73]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [74]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [75]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The response is acceptable.

Feedback: The Agent's response is professional and engaging, effectively communicating their current job role and additional endeavors. It highlights not only the intern position at Procter & Gamble but also the entrepreneurial aspect of co-founding a startup, which adds depth to their profile. Mentioning the Master's degree provides context and credibility. Additionally, inviting the User to ask specific questions encourages further engagement. Overall, the response aligns well with the intended professional tone and content.
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The Agent's reply uses a form of Pig Latin ("Odaytay, Iway otnay oldhay a atentpay...") which is unprofessional and not appropriate for a conversation with a potential client or employer. In a professional context, it’s important to provide